<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-of-Incentives" data-toc-modified-id="Analysis-of-Incentives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis of Incentives</a></span><ul class="toc-item"><li><span><a href="#Set-Up" data-toc-modified-id="Set-Up-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Set Up</a></span><ul class="toc-item"><li><span><a href="#Imports-and-Functions" data-toc-modified-id="Imports-and-Functions-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Imports and Functions</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Estimating-Cost-of-Installation" data-toc-modified-id="Estimating-Cost-of-Installation-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Estimating Cost of Installation</a></span></li><li><span><a href="#When-is-PV-Installation-Economical?" data-toc-modified-id="When-is-PV-Installation-Economical?-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>When is PV Installation Economical?</a></span></li></ul></li><li><span><a href="#State-&amp;-Federal-Tax-Incentives" data-toc-modified-id="State-&amp;-Federal-Tax-Incentives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>State &amp; Federal Tax Incentives</a></span></li></ul></li></ul></div>

## Analysis of Incentives

Two incentives currently exist for building ownders in Massachusetts to install solar panels on their properties: 

- A 15% state tax credit or \$1,000, whichever value is less, towards the cost of installation; and
- A 26% federal tax credit towards the cost of installation (declining to 22% in 2023 before expiring at the end of the year)

Other options that have so far gone unimplemented in the state include, for example:

- Ongoing (annual) payments or "credits" for the carbon emissions avoided through the use of solar energy to power a building;
    - \$3.00 per Metric ton of CO2e (MTCO2e)
    - \$45.00 per MTCO2e
    - \$120.00 per MTCO2e
- Solar net-metering credits by a state-bsaed utility, for example Eversource

Further permutations exist. For example, the annual payments for carbon emissions avoided, if coming from the state, could have a limited budget (say, \$1 million). On the other hand, perhaps these are executed through a carbon offset exchange service, which would have no budget. 

This analysis will compare the effects of the following incentive programs on rooftop solar adoption:

- The current state tax credit (15\%)
- The current federal tax credit (26\%)
- The 2023 federal tax credit (22\%)
- The 2024 federal tax credit (0\%)
- Annual payments with a budget cap of \$1 million:
    - \$3.00 per MTCO2e
    - \$45.00 per MTCO2e
    - \$120.00 per MTCO2e
- Annual payments without a budget cap (same rates)

### Set Up

#### Imports and Functions 

In [2]:
# Define functions for data & viz output locations

def dataDir(x):
    return '/home/lucia/bu/year4/semester1/EE508/project/ma-solar/data/analysisReady/' + x

def outputDir(x):
    return '/home/lucia/bu/year4/semester1/EE508/project/ma-solar/output/' + x

In [3]:
# Import libraries

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
#from rasterstats import zonal_stats
#from rasterio import open as r_open
#from rasterio.plot import show as r_show 
#from subprocess import Popen                      need these? delete if not

%matplotlib inline

#### Load Data

In [19]:
data         = gpd.read_file(dataDir('roofsSolarIncomeTaxesEmissions.shp'))
townOutlines = gpd.read_file(dataDir('isolMA.shp'))
existPV      = pd.read_csv(dataDir('existingPVSummary.csv'), index_col=0)

#### Estimating Cost of Installation

In [20]:
mean = (2.71 + 1.72) / 2

cond1 = (data['kW'] > 3) & (data['kW'] <= 7)
cond2 = (data['kW'] > 7) & (data['kW'] <= 100)
cond3 = (data['kW'] > 100) & (data['kW'] <= 2000)
cond4 = (data['kW'] <= 3) | (data['kW'] > 2000)

values = [2.71, round(mean, 2), 1.72, 0.00]

data['cost/W'] = np.select([cond1, cond2, cond3, cond4], values,
                                      default=0)

In [21]:
data['PVcostEst'] = (data['kW']*1000) * data['cost/W']
data['PVcostEst'] = round(data['PVcostEst'], 2)

data

,STRUCT_ID,SOURCE,TOWN_ID,TOWN_ID2,LOCAL_ID,AREA_SQ_FT,AREA_SQMI,index,CITY_TOWN,COUNTY,...,potent_kW,kW,wAvgInc_l,wAvgInc_u,MAIncomeT,fedIncomeT,emAvo_lbs,geometry,cost/W,PVcostEst
0,228564_891904,City of Boston,35,0,Bos_2003474000_B1,524.705830,5.350249,West Roxbury,BOSTON,SUFFOLK,...,8.046628,7.161499,122698.96,143978.48,6134.95,29447.75,6.728828e+07,"POLYGON ((-71.15372 42.27713, -71.15366 42.277...",2.21,15826.91
1,228744_891768,City of Boston,35,0,Bos_2003510000_B0,1235.985434,5.350249,West Roxbury,BOSTON,SUFFOLK,...,18.959242,16.873725,122698.96,143978.48,6134.95,29447.75,1.585428e+08,"POLYGON ((-71.15152 42.27592, -71.15150 42.275...",2.21,37290.93
2,228721_891681,City of Boston,35,0,Bos_2003527000_B0,1281.687169,5.350249,West Roxbury,BOSTON,SUFFOLK,...,19.660278,17.497647,122698.96,143978.48,6134.95,29447.75,1.644051e+08,"POLYGON ((-71.15184 42.27513, -71.15183 42.275...",2.21,38669.80
3,229467_887834,City of Boston,35,0,Bos_1812932003_B0,1107.685557,4.440987,Hyde Park,BOSTON,SUFFOLK,...,16.965452,15.099252,114557.15,131788.90,5727.86,27493.72,1.418701e+08,"POLYGON ((-71.14295 42.24045, -71.14295 42.240...",2.21,33369.35
4,229842_887062,City of Boston,35,0,Bos_1812996000_B0,1235.355673,4.440987,Hyde Park,BOSTON,SUFFOLK,...,18.901736,16.822545,99979.08,118817.14,4998.95,23994.98,1.580619e+08,"POLYGON ((-71.13836 42.23341, -71.13835 42.233...",2.21,37177.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65244,232602_889575,MAGIS,189,0,None,3318.360000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,50.862990,45.268061,118273.92,133574.65,5913.70,28385.74,4.253314e+08,"POLYGON ((-71.10502 42.25603, -71.10494 42.256...",2.21,100042.42
65245,233037_889542,MAGIS,189,0,None,2043.410000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,31.328788,27.882622,97230.77,118835.62,4861.54,23335.38,2.619806e+08,"POLYGON ((-71.09964 42.25556, -71.09974 42.255...",2.21,61620.59
65246,233261_889588,MAGIS,189,0,None,1100.700000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,16.879788,15.023011,97230.77,118835.62,4861.54,23335.38,1.411538e+08,"POLYGON ((-71.09689 42.25599, -71.09698 42.256...",2.21,33200.85
65247,234126_889280,MAGIS,189,0,None,1562.920000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,23.962098,21.326267,126930.05,138403.67,6346.50,30463.21,2.003782e+08,"POLYGON ((-71.08648 42.25316, -71.08655 42.253...",2.21,47131.05


#### When is PV Installation Economical?

In [22]:
# Compute the mean cost of installation in each town, from existing PV data

meanCostNewton = existPV.loc['Newton', 'avgFinalCost']
meanCostMilton = existPV.loc['Milton', 'avgFinalCost']
meanCostWR     = existPV.loc['West Roxbury', 'avgFinalCost']
meanCostHP     = existPV.loc['Hyde Park', 'avgFinalCost']

In [14]:
# Does this make sense for the rooftop data?
# For example, are there any for whom the mean cost is still >= annual income?

print(len(data[data['wAvgInc_l'].le(meanCostNewton) & 
               data['index'].eq('Newton')]))

print(len(data[data['wAvgInc_l'].le(meanCostMilton) & 
               data['index'].eq('Milton')]))

print(len(data[data['wAvgInc_l'].le(meanCostWR) & 
               data['index'].eq('West Roxbury')]))

print(len(data[data['wAvgInc_l'].le(meanCostHP) & 
               data['index'].eq('Hyde Park')]))

# Yes: 6 in West Roxbury and 46 in Hyde Park (none in Newton or Milton)

0
0
6
46


In [23]:
# Locate those roofs

iWR  = data['wAvgInc_l'].le(meanCostWR)
iWR &= data['index'].eq('West Roxbury')

iHP  = data['wAvgInc_l'].le(meanCostHP)
iHP &= data['index'].eq('Hyde Park')

notEcon = data.loc[iWR | iHP]
notEcon

,STRUCT_ID,SOURCE,TOWN_ID,TOWN_ID2,LOCAL_ID,AREA_SQ_FT,AREA_SQMI,index,CITY_TOWN,COUNTY,...,potent_kW,kW,wAvgInc_l,wAvgInc_u,MAIncomeT,fedIncomeT,emAvo_lbs,geometry,cost/W,PVcostEst
66,230358_891515,City of Boston,35,0,None,66.706708,4.440987,Hyde Park,BOSTON,SUFFOLK,...,1.023239,0.910683,0.0,0.0,0.0,0.0,8.556628e+06,"POLYGON ((-71.13197 42.27352, -71.13199 42.273...",0.00,0.00
128,229996_889080,City of Boston,35,0,Bos_1812172000_B8,768.752695,4.440987,Hyde Park,BOSTON,SUFFOLK,...,11.780273,10.484443,0.0,0.0,0.0,0.0,9.851013e+07,"POLYGON ((-71.13643 42.25162, -71.13648 42.251...",2.21,23170.62
157,229386_891779,City of Boston,35,0,None,7833.626557,5.350249,West Roxbury,BOSTON,SUFFOLK,...,120.102197,106.890955,0.0,0.0,0.0,0.0,1.004330e+09,"POLYGON ((-71.14360 42.27586, -71.14360 42.275...",1.72,183852.44
1015,229894_888818,City of Boston,35,0,Bos_1812172000_B0,36240.718972,4.440987,Hyde Park,BOSTON,SUFFOLK,...,555.067435,494.010017,0.0,0.0,0.0,0.0,4.641638e+09,"POLYGON ((-71.13739 42.24954, -71.13735 42.249...",1.72,849697.23
1044,230010_889056,City of Boston,35,0,Bos_1812172000_B6,1373.677764,4.440987,Hyde Park,BOSTON,SUFFOLK,...,21.050071,18.734563,0.0,0.0,0.0,0.0,1.760269e+08,"POLYGON ((-71.13638 42.25146, -71.13637 42.251...",2.21,41403.38
1045,229814_888968,City of Boston,35,0,Bos_1812172000_B7,1013.932292,4.440987,Hyde Park,BOSTON,SUFFOLK,...,15.533450,13.824771,0.0,0.0,0.0,0.0,1.298953e+08,"POLYGON ((-71.13863 42.25062, -71.13872 42.250...",2.21,30552.74
1105,230397_891439,City of Boston,35,0,Bos_1806105000_B1,314.517947,4.440987,Hyde Park,BOSTON,SUFFOLK,...,4.824508,4.293812,0.0,0.0,0.0,0.0,4.034397e+07,"POLYGON ((-71.13147 42.27283, -71.13148 42.272...",2.71,11636.23
1407,229388_891738,City of Boston,35,0,None,10380.792114,5.350249,West Roxbury,BOSTON,SUFFOLK,...,159.154375,141.647393,0.0,0.0,0.0,0.0,1.330896e+09,"POLYGON ((-71.14371 42.27540, -71.14374 42.275...",1.72,243633.52
2124,230313_891149,City of Boston,35,0,None,258.548293,4.440987,Hyde Park,BOSTON,SUFFOLK,...,3.963965,3.527928,0.0,0.0,0.0,0.0,3.314784e+07,"POLYGON ((-71.13251 42.27023, -71.13251 42.270...",2.71,9560.69
3316,230038_888735,City of Boston,35,0,None,114.478058,4.440987,Hyde Park,BOSTON,SUFFOLK,...,1.753360,1.560491,0.0,0.0,0.0,0.0,1.466212e+07,"POLYGON ((-71.13598 42.24848, -71.13600 42.248...",0.00,0.00


In [24]:
# Many of these rooftops are actually too small to merit intalling a PV
# system (<1 kW of production)

notEcon = notEcon[notEcon['kW'].ge(1.00)]

# And some of those remaining smaller than the minimum size recorded in
# the cost estimates from NREL - these won't be useful

notEcon = notEcon[notEcon['PVcostEst'].gt(0.00)]

# Still leaves 27: what are the incomes?

notEcon[['wAvgInc_l', 'PVcostEst']] 

# These are the roofs in the blocks that had no income data - filled with 0.0
# (they won't be useful for analysis either)

,wAvgInc_l,PVcostEst
128,0.0,23170.62
157,0.0,183852.44
1015,0.0,849697.23
1044,0.0,41403.38
1045,0.0,30552.74
1105,0.0,11636.23
1407,0.0,243633.52
2124,0.0,9560.69
5424,0.0,174308.76
6926,0.0,21362.81


In [25]:
# Establish a general threshold for the % of income that a system would cost
# in order for adoption to be economical, based on existing installation data

avgIncomeHP = data[data['index'].eq('Hyde Park')]['wAvgInc_l'].mean()
avgIncomeWR = data[data['index'].eq('West Roxbury')]['wAvgInc_l'].mean()
avgIncomeM  = data[data['index'].eq('Milton')]['wAvgInc_l'].mean()
avgIncomeN  = data[data['index'].eq('Newton')]['wAvgInc_l'].mean()

print(round((meanCostHP / avgIncomeHP), 2) * 100)
print(round((meanCostWR / avgIncomeWR), 2) * 100)
print(round((meanCostMilton / avgIncomeM), 2) * 100)
print(round((meanCostNewton / avgIncomeN), 2) * 100)

print('mean %: ' + str((33 + 26 + 29 + 23) / 4))

33.0
26.0
28.999999999999996
23.0
mean %: 27.75


Obviously, there are multiple criteria for when rooftop PV installation is both practical and economical:

1. At least 1 kW of estimated yearly production
2. The final cost is not greater than ~ 28% of annual income (from estimates using existing installations in each town)

### State & Federal Tax Incentives

In [26]:
data.columns

Index(['STRUCT_ID', 'SOURCE', 'TOWN_ID', 'TOWN_ID2', 'LOCAL_ID', 'AREA_SQ_FT',
       'AREA_SQMI', 'index', 'CITY_TOWN', 'COUNTY', 'TRACTA', 'BLKGRPA',
       'NAME_E', 'NAME_M', 'GHIkWh/m^2', 'area_sq_m', 'potent_kWh',
       'potent_kW', 'kW', 'wAvgInc_l', 'wAvgInc_u', 'MAIncomeT', 'fedIncomeT',
       'emAvo_lbs', 'geometry', 'cost/W', 'PVcostEst'],
      dtype='object')

In [28]:
# MA tax credit: 15% (up to $1000)

cond1 = (data['MAIncomeT'] * 0.15) < 1000
cond2 = (data['MAIncomeT'] * 0.15) >= 1000

values = [data['MAIncomeT'] * 0.15, 1000]

data['incentMA'] = np.select([cond1, cond2], values, default=0)

In [29]:
# Current federal tax credit: 26%
data['incentFed22'] = data['fedIncomeT'] * 0.26

# 2023 federal tax credit: 22%
data['incentFed23'] = data['fedIncomeT'] * 0.22

# 2024 federal tax credit: 0%
data['incentFed24'] = data['fedIncomeT'] * 0

In [30]:
# Final cost of installation with the incentives

inc = ['incentMA', 'incentFed22', 'incentFed23', 'incentFed24']

data['PVcostMA']       = data['PVcostEst'] - data['incentMA']
data['PVcostMA+Fed22'] = data['PVcostEst'] - (data['incentMA'] + data['incentFed22'])
data['PVcostMA+Fed23'] = data['PVcostEst'] - (data['incentMA'] + data['incentFed23'])
data['PVcostMA+Fed24'] = data['PVcostEst'] - (data['incentMA'] + data['incentFed24'])

data

,STRUCT_ID,SOURCE,TOWN_ID,TOWN_ID2,LOCAL_ID,AREA_SQ_FT,AREA_SQMI,index,CITY_TOWN,COUNTY,...,cost/W,PVcostEst,incentMA,incentFed22,incentFed23,incentFed24,PVcostMA,PVcostMA+Fed22,PVcostMA+Fed23,PVcostMA+Fed24
0,228564_891904,City of Boston,35,0,Bos_2003474000_B1,524.705830,5.350249,West Roxbury,BOSTON,SUFFOLK,...,2.21,15826.91,920.2425,7656.4150,6478.5050,0.0,14906.6675,7250.2525,8428.1625,14906.6675
1,228744_891768,City of Boston,35,0,Bos_2003510000_B0,1235.985434,5.350249,West Roxbury,BOSTON,SUFFOLK,...,2.21,37290.93,920.2425,7656.4150,6478.5050,0.0,36370.6875,28714.2725,29892.1825,36370.6875
2,228721_891681,City of Boston,35,0,Bos_2003527000_B0,1281.687169,5.350249,West Roxbury,BOSTON,SUFFOLK,...,2.21,38669.80,920.2425,7656.4150,6478.5050,0.0,37749.5575,30093.1425,31271.0525,37749.5575
3,229467_887834,City of Boston,35,0,Bos_1812932003_B0,1107.685557,4.440987,Hyde Park,BOSTON,SUFFOLK,...,2.21,33369.35,859.1790,7148.3672,6048.6184,0.0,32510.1710,25361.8038,26461.5526,32510.1710
4,229842_887062,City of Boston,35,0,Bos_1812996000_B0,1235.355673,4.440987,Hyde Park,BOSTON,SUFFOLK,...,2.21,37177.83,749.8425,6238.6948,5278.8956,0.0,36427.9875,30189.2927,31149.0919,36427.9875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65244,232602_889575,MAGIS,189,0,None,3318.360000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,2.21,100042.42,887.0550,7380.2924,6244.8628,0.0,99155.3650,91775.0726,92910.5022,99155.3650
65245,233037_889542,MAGIS,189,0,None,2043.410000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,2.21,61620.59,729.2310,6067.1988,5133.7836,0.0,60891.3590,54824.1602,55757.5754,60891.3590
65246,233261_889588,MAGIS,189,0,None,1100.700000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,2.21,33200.85,729.2310,6067.1988,5133.7836,0.0,32471.6190,26404.4202,27337.8354,32471.6190
65247,234126_889280,MAGIS,189,0,None,1562.920000,13.297527,Milton,"MILTON, TOWN OF",NORFOLK,...,2.21,47131.05,951.9750,7920.4346,6701.9062,0.0,46179.0750,38258.6404,39477.1688,46179.0750


In [36]:
def selections_stateFed(data, scenario_texts):
    
    data_c = data.copy()
    
    # Select those roofs where the final cost of PV installation is less than 
    # 28% of income
    incs  = ['incentFed22', 'incentFed23', 'incentFed24']
    costs = ['PVcostMA+Fed22', 'PVcostMA+Fed23', 'PVcostMA+Fed24']
    
    N = []
    A = []
    M = []
    E = []
    C_M  = []
    C_F  = []
    C_I  = []
    mC_I = []
    
    for i in range(3):
        i_sel = data_c[costs[i]].lt(data_c['wAvgInc_l']*0.28)
        
        selected = data_c[i_sel]
    
        # Count how many roofs were selected
        n = len(selected)
        N.append(n)
    
        # Compute the total area of those roofs in square kilometers
        area = round(selected['area_sq_m'].sum() / 1000000, 2)
        A.append(area)
    
        # Compute the total MW generated by installing PV on those roofs
        MW = round(selected['kW'].sum() * 0.001, 2)
        M.append(MW)
    
        # Compute the total emissions saved by installing PV on those roofs
        # and convert to MTCO2e
        em = data['emAvo_lbs'].sum() / 2205
        em = round(em, 2)
        E.append(em)
    
        # Compute the total cost for the state
        costMA = round(selected['incentMA'].sum(), 2)
        C_M.append(costMA)
        
        # Compute the total cost for the fed. govt.
        costF = round(selected[incs[i]].sum(), 2)
        C_F.append(costF)
    
        # Compute the total cost for the building owners
        costInd = round(selected['PVcostMA'].sum(), 2)
        C_I.append(costInd)
    
        # Compute the average cost for the building owners
        meanCostInd = round(selected['PVcostMA'].mean(), 2)
        mC_I.append(meanCostInd)
    
    
    results = pd.DataFrame(zip(N, A, M, E, C_M, C_F, C_I, mC_I),
                          columns=['n', 'area_km^2', 'MW', 'avoidedMTCO2e',
                                   'costForMA', 'costForFed', 
                                   'costForIndividuals',
                                   'avgCostForIndividuals'],
                          index=scenario_texts)
    
    return results

In [37]:
selections_stateFed(data, ['MA+FedIncent22', 'MA+FedIncent23', 
                                     'MA+FedIncent24'])

,n,area_km^2,MW,avoidedMTCO2e,costForMA,costForFed,costForIndividuals,avgCostForIndividuals
MA+FedIncent22,35968,2.83,415.16,6.227738e+09,30835011.82,2.698142e+08,7.968242e+08,22153.70
MA+FedIncent23,34664,2.66,390.59,6.227738e+09,29704500.61,2.198629e+08,7.436308e+08,21452.54
MA+FedIncent24,27161,1.78,260.96,6.227738e+09,23134621.42,0.000000e+00,4.634945e+08,17064.71


In [ ]:
# TODO:
#   1. Find out why round(em, 2) is not actually rounding avoided emissions
#   2. When plotting rooftops w/ solar bc of incentives, remember to plot
#      roofs w/ costEst = 0 w/ neutral color as "no cost data" (eg. Tasmania)
#   3. net metering as another incentive program?